In [45]:
#import findspark
#findspark.init()
import pyspark


In [46]:
from pyspark.sql import SparkSession , Row
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, MapType , BooleanType
from bs4 import BeautifulSoup


In [47]:
import sys , os
!{sys.executable} -m pip install beautifulsoup4

In [48]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [49]:
os.environ["HADOOP_HOME"] = "C:/Spark/spark-3.5.5-bin-hadoop3"  
os.environ["HADOOP_OPTS"] = "-Djava.library.path=C:/Spark/spark-3.5.5-bin-hadoop3/bin"

In [50]:
spark = SparkSession.builder.appName("AllFiles_Spark_Cleansing")\
    .config("spark.executor.memory", "3g")\
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.16.0,org.apache.parquet:parquet-hadoop:1.15.1")\
    .config("spark.pyspark.python", sys.executable) \
    .config("spark.pyspark.driver.python", sys.executable) \
    .config("spark.hadoop.io.native.lib.available", "false")\
    .getOrCreate()

In [51]:
comments_schema = StructType([
    StructField("_Id", IntegerType(), False),
    StructField("_PostId", IntegerType(), True),
    StructField("_UserId", IntegerType(), True),
    StructField("_UserDisplayName", StringType(), True),
    StructField("_Score", IntegerType(), True),
    StructField("_Text", StringType(), True),
    StructField("_CreationDate", TimestampType(), True),
    StructField("_ContentLicense", StringType(), True)
])

votes_schema=StructType([
    StructField("Id", IntegerType(), False),
    StructField("PostId",IntegerType(), True),
    StructField("VoteTypeId", IntegerType(), True),
    StructField("CreationDate", TimestampType(), True),
    StructField("UserId",IntegerType(), True),
    StructField("BountyAmount", IntegerType(), True),
])

Badges_schema = StructType([
    StructField("_Class", IntegerType(), True),
    StructField("_Date", TimestampType(), True),  
    StructField("_Id", IntegerType(), False),     
    StructField("_Name", StringType(), True),
    StructField("_TagBased", BooleanType(), True),
    StructField("_UserId", IntegerType(), True)  
])
posts_schema=StructType([
    StructField("AcceptedAnswerId", IntegerType(), True),
    StructField("AnswerCount",IntegerType(), True),
    StructField("Body", StringType(), True),
    StructField("ClosedDate", TimestampType(), True),
    StructField("CommentCount",IntegerType(), True),
    StructField("CommunityOwnedDate", TimestampType(), True),
    StructField("ContentLicense", StringType(), True),
    StructField("CreationDate", TimestampType(), True),
    StructField("FavoriteCount",IntegerType(), True),
    StructField("Id",IntegerType(), False),
    StructField("LastActivityDate", TimestampType(), True),
    StructField("LastEditDate", TimestampType(), True),
    StructField("LastEditorDisplayName", StringType(), True),
    StructField("LastEditorUserId",IntegerType(), True),
    StructField("OwnerDisplayName", StringType(), True),
    StructField("OwnerUserId",IntegerType(), True),
    StructField("ParentId",IntegerType(), True),
    StructField("PostTypeId",IntegerType(), True),
    StructField("Score",IntegerType(), True),
    StructField("Tags", StringType(), True),
    StructField("Title", StringType(), True),
    StructField("ViewCount",IntegerType(), True)
])


In [52]:
Comments_df = spark.read.format("xml") \
    .option("rowTag", "row") \
    .schema(comments_schema)\
    .load(r"Dataset/Comments.xml")
badges_df = spark.read.format("xml") \
    .option("rowTag", "row") \
    .schema(Badges_schema)\
    .load(r"Dataset/Badges.xml")

votes_df = spark.read.format("xml") \
    .option("rowTag", "row")\
    .option("attributePrefix", "") \
    .schema(votes_schema) \
    .load(r"Dataset/Votes.xml")
df_posts = spark.read.format("xml") \
    .option("rowTag", "row")\
    .option("attributePrefix", "") \
    .schema(posts_schema) \
    .load(r"Dataset/Posts.xml")  
Users_df = spark.read.format("xml") \
    .option("rowTag", "row") \
    .load(r"Dataset/Users.xml")

# Comments Table

In [53]:
Comments_df.show()

+---+-------+-------+----------------+------+--------------------+--------------------+---------------+
|_Id|_PostId|_UserId|_UserDisplayName|_Score|               _Text|       _CreationDate|_ContentLicense|
+---+-------+-------+----------------+------+--------------------+--------------------+---------------+
|  1|      3|     13|            NULL|     7|Could be a poster...|2010-07-19 22:15:...|   CC BY-SA 2.5|
|  2|      5|     13|            NULL|     0|Yes, R is nice- b...|2010-07-19 22:16:...|   CC BY-SA 2.5|
|  3|      9|     13|            NULL|     1|Again- why?  How ...|2010-07-19 22:18:...|   CC BY-SA 2.5|
|  4|      5|     37|            NULL|    11|It's mature, well...|2010-07-19 22:19:...|   CC BY-SA 2.5|
|  6|     14|     23|            NULL|    10|why ask the quest...|2010-07-19 22:22:...|   CC BY-SA 2.5|
|  7|     18|     36|            NULL|     1|also the US censu...|2010-07-19 22:25:...|   CC BY-SA 2.5|
|  9|     16|     78|            NULL|     1|Andrew Gelman has..

In [54]:
Comments_df.printSchema()

root
 |-- _Id: integer (nullable = false)
 |-- _PostId: integer (nullable = true)
 |-- _UserId: integer (nullable = true)
 |-- _UserDisplayName: string (nullable = true)
 |-- _Score: integer (nullable = true)
 |-- _Text: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _ContentLicense: string (nullable = true)



**Calculate Nulls**

In [55]:
Comments_df.toPandas().isnull().sum() / Comments_df.count() * 100   ########

_Id                  0.000000
_PostId              0.000000
_UserId              1.607733
_UserDisplayName    98.391386
_Score               0.000000
_Text                0.000000
_CreationDate        0.000000
_ContentLicense      0.000000
dtype: float64

**Renaming Columns**

In [56]:
Comments_df = Comments_df \
    .withColumnRenamed("_ContentLicense", "ContentLicense") \
    .withColumnRenamed("_CreationDate", "CreationDate") \
    .withColumnRenamed("_Id", "Id") \
    .withColumnRenamed("_PostId", "PostId") \
    .withColumnRenamed("_Score", "Score") \
    .withColumnRenamed("_Text", "Text") \
    .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
    .withColumnRenamed("_UserId", "UserId")

**Dropping Columns**

In [57]:
Comments_df = Comments_df.drop('UserDisplayName')

**Formatting Date**

In [58]:
Comments_df = Comments_df.withColumn("CreationDate", to_date(col("CreationDate"),"yyyy-MM-DD"))

**Handling Nulls**

In [59]:
Comments_df = Comments_df.fillna({"UserId": -2})

In [60]:
Comments_df.printSchema()       ########

root
 |-- Id: integer (nullable = false)
 |-- PostId: integer (nullable = true)
 |-- UserId: integer (nullable = false)
 |-- Score: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- ContentLicense: string (nullable = true)



In [61]:
Comments_df.show()      #########

+---+------+------+-----+--------------------+------------+--------------+
| Id|PostId|UserId|Score|                Text|CreationDate|ContentLicense|
+---+------+------+-----+--------------------+------------+--------------+
|  1|     3|    13|    7|Could be a poster...|  2010-07-19|  CC BY-SA 2.5|
|  2|     5|    13|    0|Yes, R is nice- b...|  2010-07-19|  CC BY-SA 2.5|
|  3|     9|    13|    1|Again- why?  How ...|  2010-07-19|  CC BY-SA 2.5|
|  4|     5|    37|   11|It's mature, well...|  2010-07-19|  CC BY-SA 2.5|
|  6|    14|    23|   10|why ask the quest...|  2010-07-19|  CC BY-SA 2.5|
|  7|    18|    36|    1|also the US censu...|  2010-07-19|  CC BY-SA 2.5|
|  9|    16|    78|    1|Andrew Gelman has...|  2010-07-19|  CC BY-SA 2.5|
| 10|    23|    -2|    8|I am not sure I u...|  2010-07-19|  CC BY-SA 2.5|
| 11|    43|     5|    5|There are many R ...|  2010-07-19|  CC BY-SA 2.5|
| 12|    38|    54|    0|That's just an ex...|  2010-07-19|  CC BY-SA 2.5|
| 13|    20|    24|    2|

In [62]:
Comments_df.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Comments")

# Badges Table

In [63]:
badges_df.show()        ######

+------+--------------------+---+-------+---------+-------+
|_Class|               _Date|_Id|  _Name|_TagBased|_UserId|
+------+--------------------+---+-------+---------+-------+
|     3|2010-07-19 22:39:...|  1|Teacher|    false|      5|
|     3|2010-07-19 22:39:...|  2|Teacher|    false|      6|
|     3|2010-07-19 22:39:...|  3|Teacher|    false|      8|
|     3|2010-07-19 22:39:...|  4|Teacher|    false|     23|
|     3|2010-07-19 22:39:...|  5|Teacher|    false|     36|
|     3|2010-07-19 22:39:...|  6|Teacher|    false|     37|
|     3|2010-07-19 22:39:...|  7|Teacher|    false|     50|
|     3|2010-07-19 22:39:...|  8|Teacher|    false|     55|
|     3|2010-07-19 22:39:...|  9|Student|    false|      5|
|     3|2010-07-19 22:39:...| 10|Student|    false|      8|
|     3| 2010-07-19 22:39:08| 11|Student|    false|     13|
|     3|2010-07-19 22:39:...| 12|Student|    false|     18|
|     3|2010-07-19 22:39:...| 13|Student|    false|     23|
|     3|2010-07-19 22:39:...| 14|Student

**Calculate Nulls**

In [64]:

null_perc=badges_df.select([((count(when(col(c).isNull(),c)) / badges_df.count()) * 100).alias(c) for c in badges_df.columns]) ########

In [65]:
null_perc.show()       #######

+------+-----+---+-----+---------+-------+
|_Class|_Date|_Id|_Name|_TagBased|_UserId|
+------+-----+---+-----+---------+-------+
|   0.0|  0.0|0.0|  0.0|      0.0|    0.0|
+------+-----+---+-----+---------+-------+



**Dropping Columns**

In [66]:
badges_df.printSchema()

root
 |-- _Class: integer (nullable = true)
 |-- _Date: timestamp (nullable = true)
 |-- _Id: integer (nullable = false)
 |-- _Name: string (nullable = true)
 |-- _TagBased: boolean (nullable = true)
 |-- _UserId: integer (nullable = true)



In [67]:
Badge_Disc_Dim=badges_df.drop(*["_TagBased","_Date","_UserId",'_Id'])

In [68]:
Badge_Disc_Dim.show()

+------+-------+
|_Class|  _Name|
+------+-------+
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Teacher|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3|Student|
|     3| Editor|
|     3| Editor|
+------+-------+
only showing top 20 rows



**Renaming Columns Names for Badge Disc Table** 

In [69]:
for col_name in Badge_Disc_Dim.columns:
    Badge_Disc_Dim = Badge_Disc_Dim.withColumnRenamed(col_name, col_name.lstrip("_"))

#Badge_Disc_Dim=Badge_Disc_Dim.withColumnRenamed("Id","Badge_Desc_ID")

In [70]:
Badge_Disc_Dim.printSchema()        ###

root
 |-- Class: integer (nullable = true)
 |-- Name: string (nullable = true)



**Identfying the distinct badges**

In [71]:
Badge_Disc_Dim = Badge_Disc_Dim.select('Name','Class').dropDuplicates()

In [72]:
Badge_Disc_Dim.show()
Badge_Disc_Dim.count()


+--------------------+-----+
|                Name|Class|
+--------------------+-----+
|             p-value|    3|
|             Marshal|    1|
|               anova|    2|
|     autocorrelation|    3|
|    Stellar Question|    1|
|             Curious|    3|
|   predictive-models|    3|
|             moments|    3|
|          self-study|    3|
|       Good Question|    2|
|           Outspoken|    2|
|central-limit-the...|    3|
|        optimization|    3|
|generalized-linea...|    2|
|computational-sta...|    3|
|      standard-error|    3|
|         probability|    3|
|          covariance|    3|
|  distance-functions|    3|
|              sample|    3|
+--------------------+-----+
only showing top 20 rows



500

**Adding SK**

In [73]:
Badge_Disc_Dim = Badge_Disc_Dim.withColumn("Badge_Desc_Id", monotonically_increasing_id() + lit(1))

**Rearrange coloumns**

In [74]:
Badge_Disc_Dim = Badge_Disc_Dim.select('Badge_Desc_Id','Name','Class')

In [75]:
Badge_Disc_Dim.groupBy("Class").count().show()          #####


+-----+-----+
|Class|count|
+-----+-----+
|    1|   43|
|    3|  340|
|    2|  117|
+-----+-----+



In [76]:
Badge_Disc_Dim = Badge_Disc_Dim.withColumn("Class",
    when(col("Class") == 1, lit("Gold"))
    .when(col("Class") == 2, lit("Silver"))
    .when(col("Class") == 3, lit("Bronze")))

In [77]:
Badge_Disc_Dim.show()   ###

+-------------+--------------------+------+
|Badge_Desc_Id|                Name| Class|
+-------------+--------------------+------+
|            1|             p-value|Bronze|
|            2|             Marshal|  Gold|
|            3|               anova|Silver|
|            4|     autocorrelation|Bronze|
|            5|    Stellar Question|  Gold|
|            6|             Curious|Bronze|
|            7|   predictive-models|Bronze|
|            8|             moments|Bronze|
|            9|          self-study|Bronze|
|           10|       Good Question|Silver|
|           11|           Outspoken|Silver|
|           12|central-limit-the...|Bronze|
|           13|        optimization|Bronze|
|           14|generalized-linea...|Silver|
|           15|computational-sta...|Bronze|
|           16|      standard-error|Bronze|
|           17|         probability|Bronze|
|           18|          covariance|Bronze|
|           19|  distance-functions|Bronze|
|           20|              sam

In [78]:
Badge_Disc_Dim.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Badge_Desc_Dim")

**Formatting Date**

In [79]:

badges_df = badges_df.withColumn("AssignedDate", to_date("_Date", "yyyy-MM-dd"))

In [80]:
badges_df.printSchema()

root
 |-- _Class: integer (nullable = true)
 |-- _Date: timestamp (nullable = true)
 |-- _Id: integer (nullable = false)
 |-- _Name: string (nullable = true)
 |-- _TagBased: boolean (nullable = true)
 |-- _UserId: integer (nullable = true)
 |-- AssignedDate: date (nullable = true)



In [81]:
badges_df.show()

+------+--------------------+---+-------+---------+-------+------------+
|_Class|               _Date|_Id|  _Name|_TagBased|_UserId|AssignedDate|
+------+--------------------+---+-------+---------+-------+------------+
|     3|2010-07-19 22:39:...|  1|Teacher|    false|      5|  2010-07-19|
|     3|2010-07-19 22:39:...|  2|Teacher|    false|      6|  2010-07-19|
|     3|2010-07-19 22:39:...|  3|Teacher|    false|      8|  2010-07-19|
|     3|2010-07-19 22:39:...|  4|Teacher|    false|     23|  2010-07-19|
|     3|2010-07-19 22:39:...|  5|Teacher|    false|     36|  2010-07-19|
|     3|2010-07-19 22:39:...|  6|Teacher|    false|     37|  2010-07-19|
|     3|2010-07-19 22:39:...|  7|Teacher|    false|     50|  2010-07-19|
|     3|2010-07-19 22:39:...|  8|Teacher|    false|     55|  2010-07-19|
|     3|2010-07-19 22:39:...|  9|Student|    false|      5|  2010-07-19|
|     3|2010-07-19 22:39:...| 10|Student|    false|      8|  2010-07-19|
|     3| 2010-07-19 22:39:08| 11|Student|    false|

**Renaming Columns for Badges Fact Table**

In [82]:

badges_df = badges_df.withColumnRenamed("_Date", "date") \
                    .withColumnRenamed("_Id", "AssingingBadge_BK") \
                    .withColumnRenamed("_UserId", "User_fk") \
                    .withColumnRenamed("_TagBased", "TagBased")\
                    .withColumnRenamed("_Name", "Name")\
                    


In [83]:
badges_df = badges_df.join(
    Badge_Disc_Dim,
    on = "Name",
    how = "left"
)

In [84]:
badges_df.show()

+---------+------+--------------------+-----------------+--------+-------+------------+-------------+------+
|     Name|_Class|                date|AssingingBadge_BK|TagBased|User_fk|AssignedDate|Badge_Desc_Id| Class|
+---------+------+--------------------+-----------------+--------+-------+------------+-------------+------+
|  Student|     3|2010-07-19 22:39:...|                9|   false|      5|  2010-07-19|          262|Bronze|
|  Student|     3|2010-07-19 22:39:...|               10|   false|      8|  2010-07-19|          262|Bronze|
|  Student|     3| 2010-07-19 22:39:08|               11|   false|     13|  2010-07-19|          262|Bronze|
|  Student|     3|2010-07-19 22:39:...|               12|   false|     18|  2010-07-19|          262|Bronze|
|  Student|     3|2010-07-19 22:39:...|               13|   false|     23|  2010-07-19|          262|Bronze|
|  Student|     3|2010-07-19 22:39:...|               14|   false|     24|  2010-07-19|          262|Bronze|
|  Student|     3|2

In [85]:

badges_fact = badges_df.select("AssingingBadge_BK","User_fk","Badge_Desc_Id", "TagBased", "Assigneddate")

In [86]:
badges_fact.show()

+-----------------+-------+-------------+--------+------------+
|AssingingBadge_BK|User_fk|Badge_Desc_Id|TagBased|Assigneddate|
+-----------------+-------+-------------+--------+------------+
|                9|      5|          262|   false|  2010-07-19|
|               10|      8|          262|   false|  2010-07-19|
|               11|     13|          262|   false|  2010-07-19|
|               12|     18|          262|   false|  2010-07-19|
|               13|     23|          262|   false|  2010-07-19|
|               14|     24|          262|   false|  2010-07-19|
|               16|     59|          262|   false|  2010-07-19|
|               17|     66|          262|   false|  2010-07-19|
|               18|     69|          262|   false|  2010-07-19|
|               19|     75|          262|   false|  2010-07-19|
|                1|      5|          232|   false|  2010-07-19|
|                2|      6|          232|   false|  2010-07-19|
|                3|      8|          232

In [88]:
badges_fact.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Badges_Fact")

# Votes Table

**Calculate Nulls**

In [89]:
null_perc = votes_df.select(
    (count(when(col("UserId").isNull(), 1)) / count(lit(1)) * 100).alias("UserId_null_percentage"),
    (count(when(col("BountyAmount").isNull(), 1)) / count(lit(1)) * 100).alias("BountyAmount_null_percentage")
).collect()[0]              ##

In [90]:
null_perc

Row(UserId_null_percentage=99.59631648500809, BountyAmount_null_percentage=99.37783629937461)

**Dropping columns**

In [91]:
votes_df = votes_df.drop("UserId", "BountyAmount")

**joining votes with posts**

In [92]:
df_votes_with_owner = votes_df.join(
    df_posts.select("Id", "OwnerUserId"),
    votes_df.PostId == df_posts.Id,
    "left"
)
df_votes_with_owner = df_votes_with_owner.drop(df_posts["Id"])


In [93]:
df_votes_with_owner.show()

+---+------+----------+-------------------+-----------+
| Id|PostId|VoteTypeId|       CreationDate|OwnerUserId|
+---+------+----------+-------------------+-----------+
| 20|     1|         2|2010-07-19 03:00:00|          8|
| 26|    13|         2|2010-07-19 03:00:00|         23|
| 12|     6|         2|2010-07-19 03:00:00|          5|
| 17|     6|         2|2010-07-19 03:00:00|          5|
| 25|    16|         2|2010-07-19 03:00:00|          8|
|  1|     3|         2|2010-07-19 03:00:00|         18|
|  5|     3|         2|2010-07-19 03:00:00|         18|
| 10|     3|         2|2010-07-19 03:00:00|         18|
| 21|     3|         2|2010-07-19 03:00:00|         18|
|  3|     5|         2|2010-07-19 03:00:00|         23|
|  4|     5|         2|2010-07-19 03:00:00|         23|
| 11|     5|         2|2010-07-19 03:00:00|         23|
| 15|     5|         2|2010-07-19 03:00:00|         23|
| 22|     5|         2|2010-07-19 03:00:00|         23|
| 23|     9|         2|2010-07-19 03:00:00|     

**Renaming columns**

In [94]:
df_votes_with_owner = df_votes_with_owner.withColumnRenamed("OwnerUserId", "PostOwnerId")

**Formatting Date**

In [95]:
df_votes_with_owner = df_votes_with_owner.withColumn("CreationDate", date_format(col("CreationDate"), "yyyy-MM-dd"))


In [96]:
df_votes_with_owner.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- PostId: integer (nullable = true)
 |-- VoteTypeId: integer (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- PostOwnerId: integer (nullable = true)



In [97]:
df_votes_final = df_votes_with_owner.withColumn(
    'VoteTypeName',
    when(df_votes_with_owner['VoteTypeId'] == 1, 'AcceptedByOriginator')
    .when(df_votes_with_owner['VoteTypeId'] == 2, 'Upvote')
    .when(df_votes_with_owner['VoteTypeId'] == 3, 'Downvote')
    .when(df_votes_with_owner['VoteTypeId'] == 4, 'Offensive')
    .when(df_votes_with_owner['VoteTypeId'] == 5, 'Favorite')
    .when(df_votes_with_owner['VoteTypeId'] == 6, 'Close')
    .when(df_votes_with_owner['VoteTypeId'] == 7, 'Reopen')
    .when(df_votes_with_owner['VoteTypeId'] == 8, 'BountyStart')
    .when(df_votes_with_owner['VoteTypeId'] == 9, 'BountyClose')
    .when(df_votes_with_owner['VoteTypeId'] == 10, 'Deletion')
    .when(df_votes_with_owner['VoteTypeId'] == 11, 'Undeletion')
    .when(df_votes_with_owner['VoteTypeId'] == 12, 'Migration')
    .otherwise('Unknown') 
)


In [98]:
df_votes_final

DataFrame[Id: int, PostId: int, VoteTypeId: int, CreationDate: string, PostOwnerId: int, VoteTypeName: string]

In [ ]:
df_votes_final.show()   ###

+---+------+----------+------------+-----------+------------+
| Id|PostId|VoteTypeId|CreationDate|PostOwnerId|VoteTypeName|
+---+------+----------+------------+-----------+------------+
| 20|     1|         2|  2010-07-19|          8|      Upvote|
| 26|    13|         2|  2010-07-19|         23|      Upvote|
| 12|     6|         2|  2010-07-19|          5|      Upvote|
| 17|     6|         2|  2010-07-19|          5|      Upvote|
| 25|    16|         2|  2010-07-19|          8|      Upvote|
|  1|     3|         2|  2010-07-19|         18|      Upvote|
|  5|     3|         2|  2010-07-19|         18|      Upvote|
| 10|     3|         2|  2010-07-19|         18|      Upvote|
| 21|     3|         2|  2010-07-19|         18|      Upvote|
|  3|     5|         2|  2010-07-19|         23|      Upvote|
|  4|     5|         2|  2010-07-19|         23|      Upvote|
| 11|     5|         2|  2010-07-19|         23|      Upvote|
| 15|     5|         2|  2010-07-19|         23|      Upvote|
| 22|   

In [100]:
df_votes_final.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Votes_Fact")

**Creating Votes types Dim**

In [101]:
vote_types_data = [
    Row(VoteTypeId=1, VoteTypeName='AcceptedByOriginator', Description='Marks an answer as accepted by the question author.'),
    Row(VoteTypeId=2, VoteTypeName='Upvote', Description='Upvote given to a question or answer.'),
    Row(VoteTypeId=3, VoteTypeName='Downvote', Description='Downvote given to a question or answer.'),
    Row(VoteTypeId=4, VoteTypeName='Offensive', Description='Flags a post as offensive or spam.'),
    Row(VoteTypeId=5, VoteTypeName='Favorite', Description='Indicates a user has favorited a question.'),
    Row(VoteTypeId=6, VoteTypeName='Close', Description='Suggests that a question should be closed.'),
    Row(VoteTypeId=7, VoteTypeName='Reopen', Description='Suggests that a closed question should be reopened.'),
    Row(VoteTypeId=8, VoteTypeName='BountyStart', Description='Indicates that a bounty has been added to a question.'),
    Row(VoteTypeId=9, VoteTypeName='BountyClose', Description='Indicates that a bounty has been awarded to an answer.'),
    Row(VoteTypeId=10, VoteTypeName='Deletion', Description='Indicates that a post has been deleted.'),
    Row(VoteTypeId=11, VoteTypeName='Undeletion', Description='Indicates that a post has been undeleted.'),
    Row(VoteTypeId=12, VoteTypeName='Migration', Description='Indicates that a question has been migrated to another Stack Exchange site.')
]

# Create a DataFrame
df_Vote_Types_Dim = spark.createDataFrame(vote_types_data)

In [102]:
df_Vote_Types_Dim.show()

+----------+--------------------+--------------------+
|VoteTypeId|        VoteTypeName|         Description|
+----------+--------------------+--------------------+
|         1|AcceptedByOriginator|Marks an answer a...|
|         2|              Upvote|Upvote given to a...|
|         3|            Downvote|Downvote given to...|
|         4|           Offensive|Flags a post as o...|
|         5|            Favorite|Indicates a user ...|
|         6|               Close|Suggests that a q...|
|         7|              Reopen|Suggests that a c...|
|         8|         BountyStart|Indicates that a ...|
|         9|         BountyClose|Indicates that a ...|
|        10|            Deletion|Indicates that a ...|
|        11|          Undeletion|Indicates that a ...|
|        12|           Migration|Indicates that a ...|
+----------+--------------------+--------------------+



In [103]:
df_Vote_Types_Dim.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Vote_Types_Dim")

# Posts

In [104]:
df_posts.printSchema()

root
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Id: integer (nullable = false)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)



## Filtering posts to Questions and answers 

In [105]:
df_Questions = df_posts.where(col('PostTypeId') == 1)

In [106]:
df_Answers = df_posts.where(col('PostTypeId') == 2)

## Working with Questions 

In [107]:
df_Questions_Filtered_cols = df_Questions.select('Id','OwnerUserId','CreationDate','LastActivityDate','AcceptedAnswerId','Body','Title','Tags','Score','ViewCount','AnswerCount','CommentCount')

### Dealing with nulls 

In [108]:
null_counts = df_Questions_Filtered_cols.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_Questions_Filtered_cols.columns])
#null_counts.show()

#### UserID Nulls to -2 and Accepted Answer ID nulls to -1

In [109]:
df_Questions_Handling_Nulls = df_Questions_Filtered_cols.fillna({
    'OwnerUserId':'-2',
    'AcceptedAnswerId': '-1'
})

In [110]:
#df_Questions_Handling_Nulls.show()

In [111]:
null_counts_again = df_Questions_Handling_Nulls.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_Questions_Handling_Nulls.columns])

In [112]:
#null_counts_again.show()

#### Handling Duplication 

In [113]:
duplicates = df_Questions_Handling_Nulls.groupBy('OwnerUserId','Body').count().filter("count > 1")
#duplicates.show()

In [114]:
#duplicates.count()

In [115]:
df_Questions_dropping_duplicates=df_Questions_Handling_Nulls.dropDuplicates(['OwnerUserId','Body'])

In [116]:
duplicates_validation = df_Questions_dropping_duplicates.groupBy('OwnerUserId','Body').count().filter("count > 1")
#duplicates_validation.show()

#### Converting Date types 

In [117]:
df_Questions_Date_only =  df_Questions_dropping_duplicates.withColumn("CreationDate", to_date(col("CreationDate"),"yyyy-MM-DD"))
df_Questions_Date_only = df_Questions_Date_only.withColumn("LastActivityDate", to_date(col("LastActivityDate"),"yyyy-MM-DD"))


In [118]:
#df_Questions_Date_only.collect()
#df_Questions_Date_only.limit(10).show(truncate=False)

### Handling Body (HTML to Text )

In [119]:
def html_to_text(html):
    if html is None:
        return None
    # Parse the HTML and extract the text content
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

In [120]:
html_to_text_udf = udf(html_to_text, StringType())


In [121]:
df_Questions_HTMLToText = df_Questions_Date_only.withColumn("Body", html_to_text_udf(col("Body")))


In [122]:
df_Questions_refined_text = df_Questions_HTMLToText.withColumn(
    "Body",
    trim(
        regexp_replace(
            lower(
                regexp_replace("Body", "\\s+", " ")
            ),
            "[^a-zA-Z0-9\\s]",
            ""
        )
    )
)

#### Dealing with Tags

In [123]:
df_Questions_final = df_Questions_refined_text.withColumn("Tags", expr("substring(Tags, 2, length(Tags) - 2)"))

In [124]:
df_Questions_final = df_Questions_final.withColumn("Tags", split(df_Questions_final["Tags"], "\><"))


In [125]:
df_Questions_final.select(col('Tags')).show(truncate=False)     ###

+-----------------------------------------------------------------------------------------+
|Tags                                                                                     |
+-----------------------------------------------------------------------------------------+
|[self-study, p-value, standard-deviation, least-squares, sums-of-squares]                |
|[self-study, variance, simulation, monte-carlo, numerical-integration]                   |
|[self-study, simulation, monte-carlo, importance-sampling]                               |
|[time-series, self-study, forecasting, arima]                                            |
|[probability, hypothesis-testing, self-study, normal-distribution, multivariate-analysis]|
|[hypothesis-testing, self-study, maximum-likelihood, inference, likelihood-ratio]        |
|[hypothesis-testing, self-study, inference, exponential-family]                          |
|[probability, self-study, normal-distribution, chi-squared-test, expected-value

In [126]:
df_Questions_final = df_Questions_final.withColumnRenamed("Id","QuestionId")

In [127]:
df_Questions_final.show()

+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|QuestionId|OwnerUserId|CreationDate|LastActivityDate|AcceptedAnswerId|                Body|               Title|                Tags|Score|ViewCount|AnswerCount|CommentCount|
+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|     95868|         -2|  2014-05-01|      2014-05-01|           95962|my attempt at mak...|Least Squares Fit...|[self-study, p-va...|    1|      135|          1|          19|
|    146732|         -2|  2015-04-16|      2015-04-17|          146762|i have some troub...|Variance reductio...|[self-study, vari...|    7|     1733|          1|           5|
|    151163|         -2|  2015-05-07|      2022-05-25|          151432|implement an esti...|Monte Carlo integ...|[self-s

In [128]:
df_Questions_final.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Questions")

In [129]:
df_Questions_final.printSchema()

root
 |-- QuestionId: integer (nullable = false)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- LastActivityDate: date (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)



## Working With Answers

In [130]:
df_Answers.printSchema()

root
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Id: integer (nullable = false)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)



In [131]:
df_Answers_Filtered_cols = df_Answers.select('Id','ParentId','OwnerUserId','CreationDate','LastActivityDate','Body','Score','CommentCount')

In [132]:
df_Answers_Filtered_cols.show()         ###

+---+--------+-----------+--------------------+--------------------+--------------------+-----+------------+
| Id|ParentId|OwnerUserId|        CreationDate|    LastActivityDate|                Body|Score|CommentCount|
+---+--------+-----------+--------------------+--------------------+--------------------+-----+------------+
|  5|       3|         23|2010-07-19 22:14:...|2010-07-19 22:21:...|<p>The R-project<...|   90|           3|
|  9|       3|         50|2010-07-19 22:16:...|2010-07-19 22:16:...|<p><a href="http:...|   15|           3|
| 12|       7|          5|2010-07-19 22:18:...|2010-07-19 22:18:...|<p>See my respons...|   24|           1|
| 13|       6|         23|2010-07-19 22:18:...|2010-07-19 22:18:...|<p>Machine Learni...|   27|           6|
| 14|       3|         36|2010-07-19 22:19:...|2010-07-19 22:19:...|<p>I second that ...|    6|           1|
| 15|       1|          6|2010-07-19 22:19:...|2010-07-19 22:19:...|<p>John Cook give...|   24|           0|
| 16|       3|     

### Dealing with nulls

In [133]:
null_counts = df_Answers_Filtered_cols.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_Answers_Filtered_cols.columns])


In [134]:
#null_counts.show()

In [135]:
df_Answers_Handling_Nulls = df_Answers_Filtered_cols.fillna({
    'OwnerUserId':'-2',
})

In [136]:
#null_counts = df_Answers_Handling_Nulls.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_Answers_Handling_Nulls.columns])
#null_counts.show()

### Handling Duplicates 

In [137]:
duplicates = df_Answers_Handling_Nulls.groupBy('OwnerUserId','Body','ParentId').count().filter("count > 1")


In [138]:
df_Answers_Handling_Nulls.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- ParentId: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Body: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)



In [139]:
df_Answers_Date_only =  df_Answers_Handling_Nulls.withColumn("CreationDate", to_date(col("CreationDate"),"yyyy-MM-DD"))
df_Answers_Date_only = df_Answers_Date_only.withColumn("LastActivityDate", to_date(col("LastActivityDate"),"yyyy-MM-DD"))

In [140]:
df_Answers_Date_only.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- ParentId: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- LastActivityDate: date (nullable = true)
 |-- Body: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)



### Handling Body (HTML to Text )

In [141]:
df_Answers_HTMLToText = df_Answers_Date_only.withColumn("Body", html_to_text_udf(col("Body")))


In [142]:
df_Answers_refined_text = df_Answers_HTMLToText.withColumn(
    "Body",
    trim(
        regexp_replace(
            lower(
                regexp_replace("Body", "\\s+", " ")
            ),
            "[^a-zA-Z0-9\\s]",
            ""
        )
    )
)

In [143]:
df_Answers_refined_text.select('Body').show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [144]:
df_Answers_final = df_Answers_refined_text.withColumnRenamed("Id","AnswerId")


In [145]:
df_Answers_final.coalesce(1).write.mode("overwrite").parquet("SilverDataSet/Answers")

In [146]:
spark.stop()